<a href="https://colab.research.google.com/github/dansjack/ad-450/blob/main/Week_3_Web_Scraping_(dj_winter_2022).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scrape Worldometers' Coronavirus state level data

In [55]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.worldometers.info/coronavirus/country/us/#nav-yesterday'
resp = requests.get(url)
data = resp.text

soup = BeautifulSoup(data, 'html.parser')
table = soup.find(id="usa_table_countries_today")

## Load table into DataFrame, filter data 

In [56]:
import pandas as pd

df = pd.read_html(str(table))[0]

# state column should contain just the state name (no HTML)
df['USAState'] = df['USAState'].str.replace('\xa0', ' ')
df.set_index('USAState', inplace=True)

# remove whitespace char from column name
df = df.rename(columns={ "Tot\xa0Cases/1M pop": "Tot Cases/1M pop" })

# exclude the '#', 'source', and 'projections' columns 
df = df.drop(columns=['#', 'Source', 'Projections']) 

# exclude rows for country totals
df = df.drop(["USA Total", "Total:"]) 

# clean data, NewCases column contains '+', ',' which makes the column 
# impossible to sort later if not replaced
df['NewCases'] = df['NewCases'].str.replace('+', '').str.replace(',', '')
df['NewCases'] = df['NewCases'].astype(float)
df = df.fillna(0)

## Print the first few rows of data

In [48]:
df.head()

,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population
USAState,,,,,,,,,,,
California,7376878,0.0,78730.0,0.0,0.0,0.0,186699.0,1993.0,131160392.0,3319489.0,39512223.0
Texas,5864525,0.0,78270.0,0.0,0.0,0.0,202254.0,2699.0,55371082.0,1909619.0,28995881.0
Florida,5328573,0.0,63574.0,0.0,3665356.0,1599643.0,248098.0,2960.0,50280048.0,2341031.0,21477737.0
New York,4790487,0.0,63563.0,0.0,2592921.0,2134003.0,246252.0,3267.0,94956710.0,4881199.0,19453561.0
Illinois,2773362,0.0,33446.0,0.0,1924335.0,815581.0,218861.0,2639.0,49488107.0,3905367.0,12671821.0


## DataFrame summary statistics 



In [49]:
summary_stats = df.describe()
summary_stats.drop(["count"])

,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population
mean,1.133527e+06,279.730159,14091.555556,1.984127,4.779435e+05,1.972560e+05,175776.793651,2055.444444,1.430574e+07,2.355562e+06,5.263912e+06
std,1.454533e+06,2220.289303,18106.483926,15.748520,7.228810e+05,3.783151e+05,88431.970228,1169.640664,2.203057e+07,2.144199e+06,7.056634e+06
min,3.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.123280e+05,0.000000,1880.000000,0.000000,0.000000e+00,0.000000e+00,152809.000000,1317.500000,2.638481e+06,1.299232e+06,8.233605e+05
50%,6.813820e+05,0.000000,8736.000000,0.000000,1.788590e+05,1.996800e+04,209194.000000,2523.000000,6.787993e+06,2.236708e+06,3.155070e+06
75%,1.448960e+06,0.000000,17832.500000,0.000000,5.812195e+05,2.135100e+05,233859.000000,2993.000000,1.713720e+07,2.888398e+06,6.780696e+06
max,7.376878e+06,17623.000000,78730.000000,125.000000,3.665356e+06,2.134003e+06,308780.000000,3602.000000,1.311604e+08,1.445150e+07,3.951222e+07


## Top 5s

### New Cases

In [51]:
# Note: depending on when you look at the data, you'll see a lot of 0s
top_new_cases = df.sort_values(by=["NewCases"], ascending=False)
top_new_cases = pd.DataFrame(top_new_cases, columns = ['NewCases'])
top_new_cases.head()

,NewCases
USAState,
Pennsylvania,17623.0
California,0.0
Maine,0.0
Oregon,0.0
New Mexico,0.0


### Total Deaths

In [57]:
top_total_deaths = df.sort_values(by=["TotalDeaths"], ascending=False)
top_total_deaths = pd.DataFrame(top_total_deaths, columns = ['TotalDeaths'])
top_total_deaths.head()

,TotalDeaths
USAState,
California,78730.0
Texas,78270.0
Florida,63574.0
New York,63563.0
Pennsylvania,39487.0


### Total Cases / 1m Population

In [53]:
top_cases_per_m = df.sort_values(by=["Tot Cases/1M pop"], ascending=False)
top_cases_per_m = pd.DataFrame(top_cases_per_m, columns = ['Tot Cases/1M pop'])
top_cases_per_m.head()

,Tot Cases/1M pop
USAState,
Rhode Island,308780.0
North Dakota,272944.0
Alaska,257827.0
Utah,257642.0
Tennessee,251062.0


### Total Deaths / 1m Population

In [54]:
top_deaths_per_m = df.sort_values(by=["Deaths/1M pop"], ascending=False)
top_deaths_per_m = pd.DataFrame(top_deaths_per_m, columns = ['Deaths/1M pop'])
top_deaths_per_m.head()

,Deaths/1M pop
USAState,
Mississippi,3602.0
Arizona,3504.0
New Jersey,3447.0
Alabama,3431.0
Louisiana,3296.0
